# COAWST Explorer
The Coupled Ocean Atmosphere Wave and Sediment Transport (COAWST) model is a two-way coupled framework. Here we investigate the COAWST forecast for the East and Gulf Coasts, visualizing the data using the [holoviz](holoviz.org) tools. 

In [ ]:
import xarray as xr
import cf_xarray 
import intake
import numpy as np

In [ ]:
url = 'https://mghp.osn.xsede.org/rsignellbucket1/catalogs/ioos_intake_catalog.yml'

In [ ]:
cat = intake.open_catalog(url)
list(cat)

In [ ]:
cat['COAWST-Archive']

In [ ]:
ds = cat['COAWST-Archive'].to_dask()

Find data variables that have a time dimension (but are not time `bounds` variables)

In [ ]:
time_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and not 'bounds' in var:
            time_vars.append(var)

In [ ]:
init_var = 'Hwave'

In [ ]:
ds

Import the [holoviz](https://holoviz.org) tools we need

In [ ]:
import hvplot.xarray
from geoviews import tile_sources as gvts
import panel as pn

Create widget for variable selection

In [ ]:
var_select = pn.widgets.Select(name='COAWST Variables:', options=time_vars, 
                               value=init_var)

Create widget for basemap selection

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)

The `plot` function below creates the `hvplot` panel layout object.   ROMS is on a C-Grid and variables have different coordinates depending on where they are on the grid.   So we use `cf-xarray` to determine the time, depth, longitude and latitude coordinates for each variable.  

We specify a basemap, pick the `quadmesh` plot type for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. 

We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  

We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the time dimension (and vertical dimension if it exists) so that specific values are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [ ]:
da = ds[init_var].cf.isel(T=slice(-40,-1)).unify_chunks()

In [ ]:
len(da.shape)

In [ ]:
time_vals = 48

In [ ]:
@pn.depends(var_select, base_map_select)
def plot(var, base_map):
    extra_dims = list(ds[var].dims[:-2])
    da = ds[var].cf.isel(T=slice(-time_vals,-1)).unify_chunks()
    if len(da.shape) == 4:
        mesh = da.hvplot.quadmesh(x=da.cf['longitude'].name, y=da.cf['latitude'].name, 
                              rasterize=True, geo=True, title=var, attr_labels=False, 
                              fields={da.cf['Z'].name: {'default': float(da.cf['Z'].values[-1])}},
                              groupby=extra_dims, cmap='turbo', width=600, grid=True,
                              height=600).opts(alpha=0.7, data_aspect=None, active_tools=['wheel_zoom', 'pan'])
    else:
        mesh = da.hvplot.quadmesh(x=da.cf['longitude'].name, y=da.cf['latitude'].name, 
                              rasterize=True, geo=True, title=var, attr_labels=False, 
                              groupby=extra_dims, cmap='turbo', width=600, grid=True,
                              height=600).opts(alpha=0.7, data_aspect=None, active_tools=['wheel_zoom', 'pan'])
    return pn.panel(mesh * base_map, widgets={k: pn.widgets.Select for k in extra_dims})

In [ ]:
col = pn.Column(var_select, base_map_select, plot)

We use `.servable()` below not only to display the panel object, but to make the panel servable outside the notebook via:

`panel serve COAWST-Explorer.ipynb`

In [ ]:
col.servable('COAWST Dashboard')